In [3]:
import zipfile
import os
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array, load_img
from keras.models import Model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from random import choice
from google.colab import files


In [4]:
zip_ref=zipfile.ZipFile('input.zip',r'r')
zip_ref.extractall('handwritten')
zip_ref.close()


In [5]:
from tensorflow.python.distribute.parameter_server_strategy_v2 import ParameterServerStrategyV2Extended
import os
import csv
from shutil import copyfile
import shutil
csv_file_path = '/content/handwritten/english.csv'
images_and_labels = []
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if len(row) == 2:
            image_file = row[0].strip()
            label = row[1].strip()
            images_and_labels.append((image_file, label))

In [6]:
images_and_labels.sort(key=lambda x: x[1])
base_output_directory = '/content/input'
for image_file, label in images_and_labels:
    output_directory = os.path.join(base_output_directory, label)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    source_path = os.path.join('/content/handwritten', image_file)
    destination_path = os.path.join(output_directory, image_file[4:])
    try:
      shutil.copy(source_path,destination_path)
    except:
      pass

In [7]:
data_gen=ImageDataGenerator(rescale=1./255)
train_gen=data_gen.flow_from_directory(
    '/content/input',
    target_size=(150,150),
    batch_size=10,
    class_mode='categorical'
)

Found 3410 images belonging to 63 classes.


In [9]:
import pickle
with open('class_indices.pkl', 'wb') as f:
    pickle.dump(train_gen.class_indices, f)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(63, activation='softmax')
])

In [ ]:
model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_gen,
          steps_per_epoch=8,
          epochs=4,
          verbose=1)

Epoch 1/4
8/8 [==============================] - 2s 184ms/step - loss: 0.2109 - accuracy: 0.9500
Epoch 2/4
8/8 [==============================] - 2s 190ms/step - loss: 0.2804 - accuracy: 0.9000
Epoch 3/4
8/8 [==============================] - 2s 208ms/step - loss: 0.3198 - accuracy: 0.8875
Epoch 4/4
8/8 [==============================] - 2s 206ms/step - loss: 0.2342 - accuracy: 0.9375


In [ ]:
upload=files.upload()
for fn in upload:
  loc=os.getcwd()+'/'+fn
  img=load_img(loc, target_size=(150,150))
  img=img_to_array(img)
  img=img/255.0
  img=img.reshape((1,)+img.shape)
  cls=model.predict(img)
  idx=cls[0].tolist()
  a=idx.index(max(idx))
  b= train_gen.class_indices
  for i,j in enumerate(b):
    if i==a:
      print(j)

In [ ]:
saved_model_path = 'content/pravin.bin'
tf.saved_model.save(model, saved_model_path)

In [ ]:
model.save('model.keras')
model_json = model.to_json()
with open('model_architecture.json', 'w') as json_file:
    json_file.write(model_json)